In [1]:
import pandas as pd
import os
from thefuzz import fuzz, process
import unicodedata

## Util functions


### Seasons with id_dict


In [6]:
def save_fpl_players(fpl_subfolders, season):
    for folder in fpl_subfolders:
        player_name = folder.split('/')[4].split('_')
        clean_player_name = " ".join(player_name[0:-1])
        player_df = pd.read_csv(str(folder+'/gw.csv'))

        player_dir = './data/joint/'+str(season)+'/fpl/'
         # Check if player_dir exists
        if not os.path.exists(player_dir):
            os.makedirs(player_dir)
        player_df.to_csv(player_dir + str(clean_player_name) + '.csv', index_label=False)

    print('successfully cleaned 20'+ season +' fpl data')

# Understat Files
def save_under_players(understat_files, season):
    for file_ in understat_files:
        player_name = file_.split('/')[4].split('_')
        clean_player_name = " ".join(player_name[0:-1])
        player_df = pd.read_csv(str(file_))

        player_dir = './data/joint/'+str(season)+'/understat/'
        # Check if player_dir exists
        if not os.path.exists(player_dir):
            os.makedirs(player_dir)
        player_df.to_csv(player_dir + str(clean_player_name) + '.csv', index_label=False)

    print('successfully cleaned understat 20'+ season +' data')

def joint_players_info(fpl_player_folder_path, understat_player_folder_path, season):
    fpl_subfolders = [ f.path for f in os.scandir(fpl_player_folder_path) if f.is_dir() ]
    under_files = [ f.path for f in os.scandir(understat_player_folder_path) if f.is_file() ]

    save_fpl_players(fpl_subfolders, season)
    save_under_players(under_files, season)

    print('20'+ season +' fpl and understat data now in `joint` folder')

In [7]:
def merge_fpl_understat_data(fpl_player_folder_path, understat_player_folder_path, season):
    joint_players_info(fpl_player_folder_path, understat_player_folder_path, season)

    joint_fpl_data_path = "./data/joint/"+ season + "/fpl/"
    joint_understat_path = "./data/joint/"+ season + "/understat/"

    understat_files= next(os.walk("./data/joint/"+ season + "/understat/"), (None, None, []))[2]  # [] if no file
    fpl_files = next(os.walk( "./data/joint/"+ season +"/fpl"), (None, None, []))[2]  # [] if no file
    player_ids = pd.read_csv('./data/20'+ season +'/id_dict.csv')

    understat_names = [file_.split('.')[0] for file_ in understat_files]
    fpl_names = [file_.split('.')[0] for file_ in fpl_files]

    for name in understat_names:
        # print(name)
        player_index = player_ids.index[player_ids['Understat_Name'] == name].tolist()

        if(player_index):
            indexed_fpl_name = player_ids.loc[player_index[0], 'FPL_Name']

            if(indexed_fpl_name):
                # Use Fuzzy matching get the corresponding fpl name
                fuzzy_fpl_player_name = process.extractOne(indexed_fpl_name, fpl_names, scorer=fuzz.partial_token_sort_ratio)

                fpl_player_data = pd.read_csv(joint_fpl_data_path + str(fuzzy_fpl_player_name[0])+ '.csv')
                understat_player_data = pd.read_csv(joint_understat_path + name + '.csv')

                # Change 'kickoff_time' column name to 'date
                fpl_player_data = fpl_player_data.rename(columns={'kickoff_time': 'date'})
                # change the formats: From 2021-10-03T13:00:00Z to 2021-10-03
                fpl_player_data.date = fpl_player_data.date.apply(lambda x: x.split('T')[0])

                # Dates are of the form 2021-10-03T13:00:00Z
                fpl_dates_min = fpl_player_data['date'].min()
                fpl_dates_max = fpl_player_data['date'].max()

                # Filter out player info not in the range of dates we are dealing with
                understat_filtered = understat_player_data[(pd.to_datetime(understat_player_data['date']) > pd.to_datetime(fpl_dates_min))
                                                            & (pd.to_datetime(understat_player_data['date']) < pd.to_datetime(fpl_dates_max) )]

                # Marge fpl_player_data with understat_player_data if the dates match
                player_data_merged = fpl_player_data.merge(understat_filtered, on="date")
                if(player_data_merged.shape[0]):
                    merged_dir = './data/joint/'+ season +'/merged/'
                    if not os.path.exists(merged_dir):
                        os.makedirs(merged_dir)

                    player_data_merged.to_csv(merged_dir+ indexed_fpl_name +'.csv', index_label=False )


    print('sucessfully merged 20 '+ season +' data')

In [8]:
def add_difficulty(season):
    print('====> Starting to add difficulty features to 20'+season)
    merged = './data/joint/' + season + '/merged/'
    player_names = next(os.walk((merged), (None, None, [])))[2]
    fixtures = pd.read_csv('./data/20' + season + '/fixtures.csv')

    # for name in player_names:
    #     player  = pd.read_csv('./data/joint/' + season + '/merged/'+ name)
    #     new_col = {'team_h_difficulty':[], 'team_a_difficulty':[]}

    #     for index, row in player.iterrows():
    #         for idx, row_2 in fixtures.iterrows():
    #             if row['fixture'] == row_2['id']:
    #                 new_col['team_h_difficulty'].append(row_2['team_h_difficulty'])
    #                 new_col['team_a_difficulty'].append(row_2['team_a_difficulty'])

    #     with_new_cols = [player, pd.DataFrame(new_col)]
    #     player_with_difficult = pd.concat(with_new_cols, axis=1)
    #     new_col_dir = './data/joint/' + season +'/merged_extras/'
    #     if not os.path.exists(new_col_dir):
    #         os.makedirs(new_col_dir)
    #     player_with_difficult.to_csv(new_col_dir + name)

    # Loop over each player file in player_names
    for name in player_names:
        # Load player data
        player = pd.read_csv('./data/joint/' + season + '/merged/' + name)

        # Function to get the difficulty and was_home columns based on the fixture
        def get_fixture_info(row):
            # Filter the relevant fixture
            fixture = fixtures[fixtures['id'] == row['fixture']]
            if not fixture.empty:
                fixture = fixture.iloc[0]  # Get the first (and only) match

                # Get the team difficulties
                team_h_difficulty = fixture['team_h_difficulty']
                team_a_difficulty = fixture['team_a_difficulty']
                event = fixture['event']

                return pd.Series([team_h_difficulty, team_a_difficulty, event])
            else:
                # Return NaN if no matching fixture found
                return pd.Series([None, None, None])

        # Apply the function to each row of player
        player[['team_h_difficulty', 'team_a_difficulty', 'event']] = player.apply(get_fixture_info, axis=1)

        # Save the updated DataFrame with the new columns
        new_col_dir = './data/joint/' + season + '/merged_extras/'
        if not os.path.exists(new_col_dir):
            os.makedirs(new_col_dir)
        player.to_csv(new_col_dir + name, index=False)

    print('****> successfully added difficulty features to 20'+season)

In [7]:
merge_fpl_understat_data("./data/2021-22/players/", "./data/2021-22/understat/", "21-22")
merge_fpl_understat_data("./data/2022-23/players/", "./data/2022-23/understat/", "22-23")
merge_fpl_understat_data("./data/2023-24/players/", "./data/2023-24/understat/", "23-24")
merge_fpl_understat_data("./data/2024-25/players/", "./data/2024-25/understat/", "24-25")

successfully cleaned 2021-22 fpl data
successfully cleaned understat 2021-22 data
2021-22 fpl and understat data now in `joint` folder
sucessfully merged 20 21-22 data
successfully cleaned 2022-23 fpl data
successfully cleaned understat 2022-23 data
2022-23 fpl and understat data now in `joint` folder
sucessfully merged 20 22-23 data
successfully cleaned 2023-24 fpl data
successfully cleaned understat 2023-24 data
2023-24 fpl and understat data now in `joint` folder
sucessfully merged 20 23-24 data
successfully cleaned 2024-25 fpl data
successfully cleaned understat 2024-25 data
2024-25 fpl and understat data now in `joint` folder
sucessfully merged 20 24-25 data


### Adding fixture difficulty


In [10]:
add_difficulty('21-22')
add_difficulty('22-23')
add_difficulty('23-24')
add_difficulty('24-25')

====> Starting to add difficulty features to 2021-22
****> successfully added difficulty features to 2021-22
====> Starting to add difficulty features to 2022-23
****> successfully added difficulty features to 2022-23
====> Starting to add difficulty features to 2023-24
****> successfully added difficulty features to 2023-24
====> Starting to add difficulty features to 2024-25
****> successfully added difficulty features to 2024-25


### Add Expected points


In [11]:
def add_xP(season):
    print('================> starting season 20'+season)
    players_paths = next(os.walk('./data/joint/'+ season +'/merged_extras', [None], [None],[]))[2]
    # players_paths
    for path in players_paths:
        player = pd.read_csv('./data/joint/'+ season +'/merged_extras/'+ path)
        merged = pd.read_csv('./data/20'+ season +'/gws/merged_gw.csv')

        player = player.drop(['position'], axis=1)
        merged_player = pd.merge(player, merged[['element', 'fixture', 'xP','position']], on=['element', 'fixture'], how='left')

        # Save the updated DataFrame with the new columns
        new_col_dir = './data/joint/'+ season +'/merged_extras_xP/'
        if not os.path.exists(new_col_dir):
            os.makedirs(new_col_dir)
        merged_player.to_csv(new_col_dir + path, index=False)

    print('<<<<================ starting season 20'+season)



In [12]:
add_xP('21-22')
add_xP('22-23')
add_xP('23-24')
add_xP('24-25')


================> starting season 2021-22
<<<<================ starting season 2021-22
================> starting season 2022-23
<<<<================ starting season 2022-23
================> starting season 2023-24
<<<<================ starting season 2023-24
================> starting season 2024-25
<<<<================ starting season 2024-25


### Add rolling averages


In [ ]:
def add_rolling_avgs(season):
    print('================> starting season 20'+season)
    players_paths = next(os.walk('./data/joint/'+ season +'/merged_extras_xP', [None], [None],[]))[2]
    for path in players_paths:
        player = pd.read_csv('./data/joint/'+ season +'/merged_extras_xP/'+ path,sep=',', skipinitialspace=True)

        prev = [[0 for i in range(32)],[0 for i in range(32)],[0 for i in range(32)]]
        gwks = [1]

        def rolling(row):
            row_items = [
                row['clean_sheets'], row['creativity'], row['expected_assists'], row['expected_goal_involvements'], row['expected_goals'], row['expected_goals_conceded'],
                row['goals_conceded'], row['goals_scored'], row['influence'], row['minutes'], row['own_goals'], row['penalties_missed'], row['penalties_saved'], row['red_cards'],
                 row['saves'], row['starts'],  row['team_a_score'],  row['team_h_score'], row['total_points'], row['yellow_cards'], row['goals'], row['shots'], row['xG'], row['xA'],
                 row['assists_y'], row['key_passes'], row['npg'], row['npxG'], row['xGChain'],  row['xGBuildup'], row['xP']
                ]

            if row['event'] - gwks[0] == 0:
                del prev[2]
                prev.append(row_items)

            elif row['event'] - gwks[0] == 1:
                del prev[0]
                prev.append(row_items)
                gwks[0] = row['event']

            elif row['event'] - gwks[0] == 2:
                del prev[0]
                del prev[0]
                prev.append([0 for i in range(32)])
                prev.append(row_items)
                gwks[0] = row['event']

            else:
                del prev[0]
                del prev[0]
                del prev[0]
                prev.append([0 for i in range(32)])
                prev.append([0 for i in range(32)])
                prev.append(row_items)
                gwks[0] = row['event']

            return pd.Series([round((x+y+z)/3 ,2) for x,y,z in zip(prev[0], prev[1], prev[2])])

        player[
                [
                    'clean_sheets_3', 'creativity_3', 'expected_assists_3', 'expected_goal_involvements_3', 'expected_goals_3', 'expected_goals_conceded_3',
                    'goals_conceded_3', 'goals_scored_3', 'influence_3', 'minutes_3', 'own_goals_3', 'penalties_missed_3', 'penalties_saved_3',
                    'red_cards_3', 'saves_3', 'starts_3',  'team_a_score_3',  'team_h_score_3', 'total_points_3', 'yellow_cards_3', 'goals_3', 'shots_3', 'xG_3',
                    'xA_3', 'assists_y_3', 'key_passes_3', 'npg_3', 'npxG_3', 'xGChain_3',  'xGBuildup_3',  'xP_3'
                ]
            ] = player.apply(rolling, axis=1)
        # # Drop the rolled columns
        # player.drop([
        #             'clean_sheets', 'creativity', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded',
        #             'goals_conceded', 'goals_scored', 'influence', 'minutes', 'own_goals', 'penalties_missed', 'penalties_saved',
        #             'red_cards', 'saves', 'starts',  'team_a_score',  'team_h_score', 'total_points', 'yellow_cards', 'goals', 'shots', 'xG',
        #             'xA', 'assists_y', 'key_passes', 'npg', 'npxG', 'xGChain',  'xGBuildup',  'xP'
        #         ], axis=1  )
        # Save the updated DataFrame with the new columns
        new_col_dir = './data/joint/'+ season +'/merged_extras_rolled/'
        if not os.path.exists(new_col_dir):
            os.makedirs(new_col_dir)
        player.to_csv(new_col_dir + path, index=False)

    print('<<<<================ starting season 20'+season)

In [141]:
add_rolling_avgs('22-23')
add_rolling_avgs('23-24')
add_rolling_avgs('24-25')

================> starting season 2022-23
<<<<================ starting season 2022-23
================> starting season 2023-24
<<<<================ starting season 2023-24
================> starting season 2024-25
<<<<================ starting season 2024-25


### Add odds


In [299]:
rolled = pd.read_csv('./data/joint/24-25/merged_extras_rolled/Bruno Borges Fernandes.csv')
(rolled[['h_team' ,'a_team']])

,h_team,a_team
0,Brighton,Manchester United
1,Manchester United,Liverpool
2,Southampton,Manchester United
3,Crystal Palace,Manchester United
4,Manchester United,Tottenham


In [308]:
f  = pd.read_csv('./data/odds/E0 24-25.csv')
f[f['HomeTeam'] == 'Manchester United'][['WHH', 'HomeTeam']]

,WHH,HomeTeam


In [301]:
# Load the odds data once to avoid redundant file reads
odds = pd.read_csv('./data/odds/E0 24-25.csv')
odds = odds.rename(columns={'HomeTeam': 'h_team', 'AwayTeam': 'a_team'})

def add_odds(row, odds):
    # Filter the odds DataFrame for the matching teams
    match = odds[(odds['h_team'] == row['h_team']) & (odds['a_team'] == row['a_team'])]

    # Check if a match is found
    if not match.empty:
        # Extract the relevant odds values
        odds_ = match.iloc[0]
        WHH = odds_['WHH']
        WHD = odds_['WHD']
        WHA = odds_['WHA']
        return pd.Series([WHH, WHD, WHA])
    else:
        # Return NaN for rows with no match
        return pd.Series([None, None, None])

# Apply the function to the 'rolled' DataFrame
rolled[['WHH', 'WHD', 'WHA']] = rolled.apply(add_odds, axis=1, odds=odds)

rolled[['h_team','a_team', 'WHH','WHD', 'WHA', 'date', 'total_points']]


,h_team,a_team,WHH,WHD,WHA,date,total_points
0,Brighton,Manchester United,None,None,None,2024-08-24,2
1,Manchester United,Liverpool,None,None,None,2024-09-01,2
2,Southampton,Manchester United,None,None,None,2024-09-14,5
3,Crystal Palace,Manchester United,None,None,None,2024-09-21,3
4,Manchester United,Tottenham,None,None,None,2024-09-29,-2


In [13]:
columns = ['bonus', 'clean_sheets','expected_assists (xA)', 'expected_goal_involvements', 'expected_goals (xG)', 'expected_goals_conceded', 'ict_index', 'minutes',
            'red_cards','total_points', 'was_home', 'yellow_cards', 'shots', 'npxG', 'xGChain', 'xGBuildup', 'team_h_difficulty', 'team_a_difficulty', 'opponent_team',
             'threat', 'value', 'key_passes',
            ]

```
    Total Points – Bonus Points (tp-bp), Minutes, Yellow Cards, Red Cards, Expected Goals (xG), Expected Assists (xA), Non-penalty Expected Goals (npxG),
    Shots, Expected Goals Against, Expected_goal_involvements,  clean_sheets, ict_index, opponent_team, Expected Goals Buildup (xG Buildup), threat, value,
    Key Passes,


    Games,  Expected Goals Chain (xG Chain),  Non-penalty Expected Goal Difference (npxGD), Non-penalty Expected Goals Against (npxGA), Expected Points (xPts)
```


<!-- ### Merge data -->


In [14]:
def merge_files(season):
    paths = next(os.walk('./data/joint/'+ season +'/merged_extras_xP', [None], [None],[]))[2]
    files_list = [pd.read_csv('./data/joint/'+ season +'/merged_extras_xP/' + path)  for  path in paths ]
    merged_files = pd.concat(files_list)

    # Save the new DataFrame
    new_col_dir = './data/joint/'+ season +'/'
    print(new_col_dir)

    merged_files.to_csv(new_col_dir  +'merged_player_data.csv', index=False)
    # print('<<<<================ starting season 20'+ season)


In [15]:
merge_files('21-22')
merge_files('22-23')
merge_files('23-24')
merge_files('24-25')

./data/joint/21-22/
./data/joint/22-23/
./data/joint/23-24/
./data/joint/24-25/


In [12]:
pd.read_csv('./data/joint/24-25/merged_player_data.csv')[ 'round']

0       2
1       5
2       3
3       4
4       5
       ..
1040    5
1041    6
1042    2
1043    3
1044    5
Name: round, Length: 1045, dtype: int64